In [1]:
import re
import pandas as pd
import json

import matplotlib.pyplot as plt
import seaborn as sns

from google.cloud import bigquery
import warnings
warnings.filterwarnings('ignore')
import matplotlib.dates as mdates
from statsmodels.tsa.seasonal import seasonal_decompose
from decimal import Decimal

pd.set_option('display.max_colwidth', None)    # Show full width of each column
pd.set_option('display.expand_frame_repr', False)

In [2]:
# !pip install openpyxl

In [3]:
gcs_path = "gs://trd-sf-ntb"
target="units"
brand = 'LACOSTE'
experiment_name = "model_pipeline"  # or something like f"exp_{datetime.now():%Y%m%d_%H%M%S}"
cutoff_month='2025-07'

target_month='2026-10'
marketing_feature_start_dt='2023-03-01'
file_output = "monthly_features_mmm_faces.parquet"
 

#### Online

In [4]:
client = bigquery.Client()

from google.cloud import bigquery

def fetch_bigquery_data(
    project_id: str = 'chb-svc-tredence-d001',
    dataset_table: str = 'shared_analytics_prod.mkting_brands_union',
    brand: str = 'LACOSTE',
    start_date: str = marketing_feature_start_dt ,
    end_date: str = None,
    limit: int = None
) -> pd.DataFrame:
    """
    Fetches marketing data from BigQuery for a specific brand and date range.
    
    Args:
        project_id (str): GCP project ID. Default: 'chb-svc-tredence-d001'
        dataset_table (str): Dataset and table name. Default: 'shared_analytics_prod.mkting_brands_union'
        brand (str): Brand to filter. Default: 'FACES'
        start_date (str): Start date in YYYY-MM-DD format. Default: '2023-03-01'
        end_date (str): End date in YYYY-MM-DD format. If None, uses current date.
        limit (int): Maximum number of rows to return. If None, returns all rows.
    
    Returns:
        pd.DataFrame: DataFrame containing the query results
    """
    client = bigquery.Client(project=project_id)
    
    query = f"""
        SELECT *
        FROM `{project_id}.{dataset_table}`
        WHERE brand = '{brand}'
        AND date >= '{start_date}'
    """
    
    if end_date:
        query += f" AND date <= '{end_date}'"
    
    if limit:
        query += f" LIMIT {limit}"
    
    query_job = client.query(query)
    return query_job.to_dataframe()

mmm = fetch_bigquery_data(
    brand='LACOSTE',
    start_date= marketing_feature_start_dt
)

In [5]:
df = mmm.copy()

In [6]:
df.head(2)

,date,brand,business_type,sales_channel,platform,source,medium,default_channel_grouping,objective_type,total_cost_usd,...,cpm,ctr,campaign_id,currency,account_id,account_name,campaign_name,vertical,sub_vertical,boat
0,2024-07-26,LACOSTE,ecommerce,web,tiktok ads,tiktok,paid_social,paid social,REACH,0.000,...,0.00,0.00,1792678559036466,USD,7070814795133485057,Lacoste ME,FPMX-Lacoste-UAE-EN-Tiktok-Awareness-Reach-Ramadan-Mar2024,MANAGED COMPANIES,FASHION & ACCESSORIES,PREMIUM & DELEGATION
1,2024-07-26,LACOSTE,ecommerce,web,google ads,google,performance_max,cross network,Maximize Conversion Value,187.555,...,2.76,0.01,21452747914,USD,7782253166,Lacoste KSA,KSA_AR_Performance Max_No Asset,MANAGED COMPANIES,FASHION & ACCESSORIES,PREMIUM & DELEGATION


In [7]:
df['date'] = pd.to_datetime(df['date'])

In [8]:
df.date.max()

Timestamp('2025-08-07 00:00:00')

In [9]:
def extract_region_from_campaign(
    df: pd.DataFrame,
    campaign_col: str = 'campaign_name',
    regions: list = ['UAE'],
    output_col: str = 'region'
) -> pd.DataFrame:
    """
    Extracts the first matching region code from a campaign name column.
    
    Args:
        df (pd.DataFrame): Input DataFrame.
        campaign_col (str): Column containing campaign names. Default: 'campaign_name'.
        regions (list): List of region codes to match. Default: ['UAE', 'UAE'].
        output_col (str): Name of the output column. Default: 'region'.
    
    Returns:
        pd.DataFrame: DataFrame with the extracted region column.
    """
    # Create regex pattern (case-insensitive)
    pattern = r'(' + '|'.join(regions) + r')'
    
    # Extract matches and take the first one (converted to uppercase)
    df[output_col] = (
        df[campaign_col]
        .str.findall(pattern, flags=re.IGNORECASE)
        .apply(lambda x: x[0].upper() if x else None)
    )
    
    return df

df = extract_region_from_campaign(df)

In [10]:
df.head(2)

,date,brand,business_type,sales_channel,platform,source,medium,default_channel_grouping,objective_type,total_cost_usd,...,ctr,campaign_id,currency,account_id,account_name,campaign_name,vertical,sub_vertical,boat,region
0,2024-07-26,LACOSTE,ecommerce,web,tiktok ads,tiktok,paid_social,paid social,REACH,0.000,...,0.00,1792678559036466,USD,7070814795133485057,Lacoste ME,FPMX-Lacoste-UAE-EN-Tiktok-Awareness-Reach-Ramadan-Mar2024,MANAGED COMPANIES,FASHION & ACCESSORIES,PREMIUM & DELEGATION,UAE
1,2024-07-26,LACOSTE,ecommerce,web,google ads,google,performance_max,cross network,Maximize Conversion Value,187.555,...,0.01,21452747914,USD,7782253166,Lacoste KSA,KSA_AR_Performance Max_No Asset,MANAGED COMPANIES,FASHION & ACCESSORIES,PREMIUM & DELEGATION,None


In [11]:
df.business_type.value_counts()

business_type
ecommerce    49697
Name: count, dtype: int64

In [12]:
def calculate_monthly_metrics(
    df: pd.DataFrame,
    brand_region_map: dict = {'LACOSTE': 'UAE'},
    metrics: list = ['total_cost_usd', 'impressions', 'clicks', 'ctr', 'cpm', 'cpc']
) -> pd.DataFrame:
    """
    Calculate monthly marketing metrics for specific brand-region combinations,
    including pivoted total_cost_usd by source.

    Args:
        df: Input DataFrame containing marketing data
        brand_region_map: Dictionary of {brand: region} pairs to analyze
        metrics: List of metrics to include in aggregation

    Returns:
        DataFrame with monthly metrics including total_cost_usd pivoted by source.
    """
    # Filter for specified brand-region pairs
    filter_condition = df.apply(
        lambda row: any(row['brand'] == brand and row['region'] == region 
                     for brand, region in brand_region_map.items()),
        axis=1
    )
    filtered_df = df[filter_condition].copy()

    # Ensure date is datetime and create month column
    filtered_df['date'] = pd.to_datetime(filtered_df['date'])
    filtered_df['month'] = filtered_df['date'].dt.to_period('M')

    # Base monthly aggregation
    monthly_features = filtered_df.groupby(['month', 'brand', 'region']).agg({
        'total_cost_usd': 'sum',
        'impressions': 'sum',
        'clicks': 'sum',
        'ctr': 'mean',
        'cpm': 'mean',
        'cpc': 'mean'
    }).reset_index()

    # Pivot total_cost_usd by source
    cost_by_source = (
        filtered_df.pivot_table(
            index=['month', 'brand', 'region'],
            columns='source',
            values='total_cost_usd',
            aggfunc='sum',
            fill_value=0
        )
        .add_prefix('total_cost_usd_')
        .reset_index()
    )

    # Merge source-specific cost into monthly features
    monthly_features = pd.merge(monthly_features, cost_by_source, on=['month', 'brand', 'region'], how='left')

    # Derived metrics
    monthly_features['CPM'] = (monthly_features['total_cost_usd'] / monthly_features['impressions']) * 1000
    monthly_features['CPC'] = monthly_features['total_cost_usd'] / monthly_features['clicks']
    monthly_features['CTR'] = (monthly_features['clicks'] / monthly_features['impressions']) * 100

    # Column ordering
    column_order = ['month', 'brand', 'region', 'total_cost_usd', 'impressions', 
                    'clicks', 'CTR', 'CPM', 'CPC', 'ctr', 'cpm', 'cpc'] + \
                   [col for col in monthly_features.columns if col.startswith('total_cost_usd_')]
    column_order = [col for col in column_order if col in monthly_features.columns]

    return monthly_features[column_order]

monthly_features_source = calculate_monthly_metrics(df)
monthly_features_source.head(2)

,month,brand,region,total_cost_usd,impressions,clicks,CTR,CPM,CPC,ctr,cpm,cpc,total_cost_usd_facebook,total_cost_usd_google,total_cost_usd_instagram,total_cost_usd_tiktok
0,2023-03,LACOSTE,UAE,28162.5903,7235037.0,109338,1.511229,3.892529,0.257574,0.085424,39.432401,0.331243,12134.6539,13778.8104,2089.6460,159.48
1,2023-04,LACOSTE,UAE,55004.4834,15989655.0,258971,1.619616,3.440004,0.212396,0.082628,41.806212,0.314515,23829.4612,23178.6936,6987.1286,1009.20


In [13]:
monthly_features_source.month.max()

Period('2025-08', 'M')

In [14]:
filtered_df = monthly_features_source[monthly_features_source['month'] <= cutoff_month]

In [15]:
filtered_df.month.max()

Period('2025-07', 'M')

In [16]:
def extend_monthly_spend_by_sply(df: pd.DataFrame, target_month) -> pd.DataFrame:
    """
    Extend monthly marketing data by copying same months from the previous year
    for each brand-region combination, until October 2026.

    Parameters:
    -----------
    df : pd.DataFrame
        Input DataFrame with columns ['month', 'brand', 'region', ...]
        where 'month' is a pandas Period ('M' frequency).

    Returns:
    --------
    pd.DataFrame
        DataFrame with original + extended synthetic data.
    """
    df = df.copy()

    if not pd.api.types.is_period_dtype(df['month']):
        df['month'] = pd.to_datetime(df['month']).dt.to_period('M')

    df['is_actual'] = True
    extended_rows = []

    target_month = pd.Period(target_month, freq='M')

    for (brand, region), group in df.groupby(['brand', 'region']):
        current_df = group.copy()
        current_max_month = current_df['month'].max()

        while current_max_month < target_month:
            months_to_extend = pd.period_range(
                start=current_max_month + 1, 
                end=min(current_max_month + 12, target_month), 
                freq='M'
            )

            source_months = months_to_extend - 12
            source_data = current_df[current_df['month'].isin(source_months)].copy()

            if source_data.empty:
                break  # No more historical data to copy

            source_data['month'] = source_data['month'] + 12
            source_data['is_actual'] = False

            extended_rows.append(source_data)
            current_df = pd.concat([current_df, source_data], ignore_index=True)
            current_max_month = source_data['month'].max()

    df_extended = pd.concat([df] + extended_rows, ignore_index=True) if extended_rows else df

    return df_extended.sort_values(['brand', 'region', 'month']).reset_index(drop=True)

In [17]:
df_extended = extend_monthly_spend_by_sply(filtered_df,target_month)
df_extended.tail(13)

,month,brand,region,total_cost_usd,impressions,clicks,CTR,CPM,CPC,ctr,cpm,cpc,total_cost_usd_facebook,total_cost_usd_google,total_cost_usd_instagram,total_cost_usd_tiktok,is_actual
31,2025-10,LACOSTE,UAE,24353.0549,5135021.0,71906,1.400306,4.742542,0.338679,0.101065,31.237903,0.281677,5453.3107,14824.3948,4075.3494,0.00,False
32,2025-11,LACOSTE,UAE,81700.4650,18118025.0,144393,0.796958,4.509347,0.56582,0.081506,36.298130,0.453974,27643.4222,27128.5252,23779.3376,3149.18,False
33,2025-12,LACOSTE,UAE,42750.9811,8131304.0,112015,1.377577,5.257580,0.381654,0.092540,25.935370,0.297138,14511.7464,19105.1805,9134.0542,0.00,False
34,2026-01,LACOSTE,UAE,40756.8988,8856281.0,122972,1.388529,4.602033,0.331432,0.117769,35.935289,0.315496,11135.0656,20275.7383,9346.0949,0.00,False
35,2026-02,LACOSTE,UAE,36843.4401,21354612.0,220876,1.034325,1.725315,0.166806,0.055152,22.028618,0.343349,13441.1581,14639.7697,7502.5123,1260.00,False
36,2026-03,LACOSTE,UAE,44396.5681,9161381.0,153872,1.679572,4.846056,0.288529,0.058571,22.869147,0.231590,12175.3327,20341.6183,11879.6171,0.00,False
37,2026-04,LACOSTE,UAE,25687.5726,12138873.0,84009,0.692066,2.116141,0.305772,0.048674,15.713573,0.254382,7564.0582,13742.6824,4380.8320,0.00,False
38,2026-05,LACOSTE,UAE,46158.5267,11473856.0,112885,0.983845,4.022931,0.408899,0.047934,16.218286,0.274418,13857.3584,20192.7671,12108.4012,0.00,False
39,2026-06,LACOSTE,UAE,42443.6952,6659561.0,47007,0.705857,6.373347,0.902923,0.054069,23.286476,0.307643,13453.3269,16221.8644,12768.5039,0.00,False
40,2026-07,LACOSTE,UAE,32105.5283,5306816.0,38879,0.732624,6.049866,0.825781,0.053453,25.452458,0.380064,6668.4610,19200.1287,6236.9386,0.00,False


In [18]:
df_extended[df_extended["month"].isin(["2026-09","2025-09"])]

,month,brand,region,total_cost_usd,impressions,clicks,CTR,CPM,CPC,ctr,cpm,cpc,total_cost_usd_facebook,total_cost_usd_google,total_cost_usd_instagram,total_cost_usd_tiktok,is_actual


In [19]:
# till_may_data = monthly_features_source[monthly_features_source['month'] <= '2025-05']

In [20]:
df_extended['date'] = df_extended['month'].dt.to_timestamp()
df_extended = df_extended.drop(columns='month')

In [21]:
df_extended.date.max()

Timestamp('2026-10-01 00:00:00')

In [22]:
df_extended.head(2)

,brand,region,total_cost_usd,impressions,clicks,CTR,CPM,CPC,ctr,cpm,cpc,total_cost_usd_facebook,total_cost_usd_google,total_cost_usd_instagram,total_cost_usd_tiktok,is_actual,date
0,LACOSTE,UAE,28162.5903,7235037.0,109338,1.511229,3.892529,0.257574,0.085424,39.432401,0.331243,12134.6539,13778.8104,2089.6460,159.48,True,2023-03-01
1,LACOSTE,UAE,55004.4834,15989655.0,258971,1.619616,3.440004,0.212396,0.082628,41.806212,0.314515,23829.4612,23178.6936,6987.1286,1009.20,True,2023-04-01


In [23]:
full_path = f"{gcs_path}/{target}/{brand}/{experiment_name}/{cutoff_month}/{file_output}"
full_path

'gs://trd-sf-ntb/units/LACOSTE/model_pipeline/2025-07/monthly_features_mmm_faces.parquet'

In [24]:
df_extended.to_parquet(full_path, index=False)

In [25]:
hello

NameError: name 'hello' is not defined

In [ ]:
def eda_features(df_input: pd.DataFrame, feature_name: str):
    '''
    feature_name can only be following : temporal, promotion, marketing, store
    '''
    print(f"------EDA on {feature_name} features------------------")
    try:
        print(f"min & max dates {df_input.date.min().date()},{df_input.date.max().date()}",end="\n\n")
    except:
        pass
    print(f"shape of dataset : {df_input.shape}",end="\n\n")
    print(f"{feature_name} features: {df_input.columns.values}",end="\n\n")
    print(f"missing values :\n{df_input.isnull().sum()}")
    

In [ ]:
eda_features(df_extended,"marketing")